# Linear Regression with One Variable

![Introduction](Images/Course Logo.PNG)

### Linear regression من المتوقع في نهاية هذا الدرس أن يكون المتعلم قادر بمفرده على بناء نظام 
### Gradient descent من أجل مجموعة بيانات بدخل و خرج وحيدين بالإضافة إلى فهم خوارزمية 
### .و التي تعتبر حجر الأساس في العديد من خوارزميات تعلم الآلة و التعلم العميق

###  خلال درس اليوم سنتعلم سوية تعتبر من أبسط خوارزميات تعلم الآلة ألا و هي خوارزمية
###  Linear Regression
###  Y و هي خوارزمية بسيطة تستخدم لإيجاد النموذج الرياضي الذي يربط بين متحول رياضي  
###  X و متحول أو عدة متحولات رياضية أخرى 
###  Y بعد إيجاد هذه العلاقة يصبح بإمكاننا التنبؤ بقيمة 
###  X من أجل قيمة معينة للمتغير/ مجموعة المتغيرات

### ملاحظة: إذا كنت قد أصبت بالإحباط من التعريف الممل في الأعلى فلا تأبه به و دعنا نبدأ بمراجعة ما كنا قد تعلمناه خلال المحاضرة 
### حيث أننا قمنا سوية ببناء نظام قادر على التنبؤ بدرجات طالب ما بالاعتماد على عدد ساعات دراسته لهذه المادة

In [111]:
from IPython.display import Image
from IPython.display import display
display(Image(url='./Images/gradient_descent_example.gif'))

In [54]:
import numpy as np # تتميز هذه المكتبة بقدرتها على التعامل مع المصفوفات الضخمة متعددة الأبعاد كما أنها مزودة
                   # بالعديد من التوابع الرياضية للتعامل مع هذه المصفوفات
import pandas as pd # تعتبر من أهم و أقوى المكتبات الموجودة بلغة بايثون لمعالجة و تحليل البيانات

## الخطوة الأولى في مشاريع تعلم الآلة هي قراءة مجموعات البيانات, كما أن هناك طرق عدة لقراءة هذه الملفات منها الطريقتين التاليتين 
## %timeit و اللتان سنقوم بقياس أداء كل منهما عبر استخدام 

In [55]:
%timeit students_data = pd.read_csv('data1.csv', header=None)

100 loops, best of 3: 5.01 ms per loop


In [56]:
%timeit test = np.genfromtxt('data1.csv',delimiter=',')


100 loops, best of 3: 5.04 ms per loop


In [57]:
students_data = pd.read_csv('data1.csv', header=None)

## .عادة ما نقوم بمعالجة البيانات و التحقق من سلامتها و خلوها من الشوائب قبل البدء بعملية التدريب
## :مع أن مجموعة البيانات المتوفرة لدينا خالية تماماً من أية مشاكل لكننا سنقوم بالتحقق من ذلك بأنفسنا كما يلي

In [58]:
students_data.head()

,0,1
0,32.502345,31.707006
1,53.426804,68.777596
2,61.530358,62.562382
3,47.475640,71.546632
4,59.813208,87.230925


In [60]:
len(students_data)

100

In [61]:
students_data.describe()

,0,1
count,100.000000,100.000000
mean,48.958341,72.735051
std,9.746379,16.658187
min,25.128485,31.707006
25%,41.526164,60.788586
50%,49.634970,72.179542
75%,56.762631,83.215080
max,70.346076,118.591217


In [62]:
students_data[students_data.isnull().values]

,0,1


In [63]:
students_data.columns = ['grade', 'hours']

In [64]:
students_data.dtypes

grade    float64
hours    float64
dtype: object

In [65]:
students_data.head()

,grade,hours
0,32.502345,31.707006
1,53.426804,68.777596
2,61.530358,62.562382
3,47.475640,71.546632
4,59.813208,87.230925


## الخطوة الثانية: لنبدأ الآن بتنجيز التابع الخاص بإيجاد قيم المربعات الصغرى لحساب الخطأ
![title](Images/linear_regression_error1.png)

In [99]:
x = np.zeros(len(students_data))
y = np.zeros(len(students_data))

def compute_error_for_line_given_data(b, m, students_data):
    error=0
    x = students_data['grade']
    y = students_data['hours']
    N = len(students_data)
    error = np.sum((y-(m*x + b))**2)/N
    return error


## الخطوة الثالثة: الآن لنقم بتنجيز تابع إيجاد قيم شعاع المشتقات الجزئية من أجل خطوة واحدة
## Step Gradient
![title](Images/linear_regression_gradient1.png)

In [105]:
def step_gradient(b_current, m_current, data, learning_rate):
    
    b = b_current
    m = m_current
    N = len(data)
    x = data['grade']
    y = data['hours']
    
    m_gradient = np.sum(-2*x*(y - (m*x + b)))/N
    b_gradient = np.sum(-2*(y - (m*x +b)))/N
    new_m = m - learning_rate*m_gradient
    new_b = m - learning_rate*b_gradient
    return [new_m, new_b]#,m_gradient,b_gradient]

![gradient_descent](Images/GradientDescent.PNG)

## الخطوة الرابعة و الأخيرة: و هي أن نقوم بإنشاء التابع المسؤول عن تنفيذ الخوارزمية 
## .(من أجل عدد معين من التكرارات (في حالتنا قمنا باختيار 1000 تكرار

In [107]:
def gradient_descent_runner(data, starting_b, starting_m, learning_rate, num_iterations):
    b = starting_b
    m = starting_m
    for i in range(num_iterations):
        b, m = step_gradient(b, m, data, learning_rate)
    return [b, m]

## التنفيذ

In [114]:
def run():
    # Hyperparameters
    learning_rate = 0.001
    num_iterations = 1000
    initial_m = 0
    initial_b = 0
    print("Starting gradient descent at b = {0}, m = {1}, error = {2}".format(initial_b, initial_m, compute_error_for_line_given_data(initial_b, initial_m, students_data)))
    print("Running...")
    [b, m] = gradient_descent_runner(students_data, initial_b, initial_m, learning_rate, num_iterations)
    print("After {0} iterations b = {1}, m = {2}, error = {3}".format(num_iterations, b, m, compute_error_for_line_given_data(b, m, students_data)))

In [115]:
run()

Starting gradient descent at b = 0, m = 0, error = 5565.10783448321
Running...
After 1000 iterations b = 1.4312175809829089, m = 1.4564184742387396, error = 111.94568519744024


## Alpha في الختام نستعرض الشرح التوضيحي التالي و الذي يبين أهمية عامل التعلم و الذي عادة ما نرمز له بالرمز اللاتيني  
![Learning_Rate](Images/learningRate.PNG)

## ------------------------------------------------------- النهاية -----------------------------------------------------------------------